<a href="https://colab.research.google.com/github/sruby8/uplift.ai/blob/master/Skeleton_with_kinematic_velo_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Multi-View 3D Skeleton Animation with Velocity Heatmaps and Chart Below the Skeleton

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
import re

# === Load Dataset ===
df = pd.read_csv("./uplift_data_export_20250427183322.csv")

# === Handedness Detection ===
pitcher_handedness = df['handedness'].dropna().iloc[0].lower() if 'handedness' in df.columns else 'left'
flip_x = pitcher_handedness == 'left'

# === Dominant Wrist Detection ===
right_cols = ['right_wrist_jc_3d_x', 'right_wrist_jc_3d_y', 'right_wrist_jc_3d_z']
left_cols = ['left_wrist_jc_3d_x', 'left_wrist_jc_3d_y', 'left_wrist_jc_3d_z']
right_vel = np.linalg.norm(np.diff(df[right_cols], axis=0, prepend=df[right_cols].iloc[[0]]), axis=1)
left_vel = np.linalg.norm(np.diff(df[left_cols], axis=0, prepend=df[left_cols].iloc[[0]]), axis=1)
thrw_joint = 'right_wrist_jc_3d' if right_vel.sum() > left_vel.sum() else 'left_wrist_jc_3d'

# === Core Joints and Mapping ===
core_joints = [
    'left_shoulder_jc_3d', 'left_elbow_jc_3d', 'left_wrist_jc_3d',
    'right_shoulder_jc_3d', 'right_elbow_jc_3d', 'right_wrist_jc_3d',
    'left_hip_jc_3d', 'left_knee_jc_3d', 'left_ankle_jc_3d',
    'right_hip_jc_3d', 'right_knee_jc_3d', 'right_ankle_jc_3d'
]

available_joints = [j for j in core_joints if all(f"{j}_{ax}" in df.columns for ax in ['x', 'y', 'z'])]
core_joints = available_joints

def get_xyz_remapped(joint):
    x = df[f"{joint}_x"].values
    y = df[f"{joint}_y"].values
    z = df[f"{joint}_z"].values
    return np.stack([-x if flip_x else x, z, y], axis=1)

# Virtual spine
spine_top = (get_xyz_remapped("left_shoulder_jc_3d") + get_xyz_remapped("right_shoulder_jc_3d")) / 2
spine_base = (get_xyz_remapped("left_hip_jc_3d") + get_xyz_remapped("right_hip_jc_3d")) / 2
core_joints += ['spine_top_virtual', 'spine_base_virtual']

skeleton_frames = [spine_top if j == 'spine_top_virtual' else spine_base if j == 'spine_base_virtual' else get_xyz_remapped(j) for j in core_joints]
skeleton_frames = np.stack(skeleton_frames, axis=1)
velocity_frames = np.linalg.norm(np.diff(skeleton_frames, axis=0, prepend=skeleton_frames[0:1]), axis=2)
velocity_normed = (velocity_frames - velocity_frames.min()) / (velocity_frames.max() - velocity_frames.min())

# Frame limits
X, Y, Z = skeleton_frames[:, :, 0], skeleton_frames[:, :, 1], skeleton_frames[:, :, 2]
x_min, x_max = X.min() - 0.3, X.max() + 0.3
y_min, y_max = Y.min() - 0.3, Y.max() + 0.3
z_min, z_max = Z.min() - 0.3, Z.max() + 0.3

# Bones
bones = [("left_shoulder_jc_3d", "left_elbow_jc_3d"), ("left_elbow_jc_3d", "left_wrist_jc_3d"),
         ("right_shoulder_jc_3d", "right_elbow_jc_3d"), ("right_elbow_jc_3d", "right_wrist_jc_3d"),
         ("left_hip_jc_3d", "left_knee_jc_3d"), ("left_knee_jc_3d", "left_ankle_jc_3d"),
         ("right_hip_jc_3d", "right_knee_jc_3d"), ("right_knee_jc_3d", "right_ankle_jc_3d"),
         ("left_shoulder_jc_3d", "right_shoulder_jc_3d"), ("left_hip_jc_3d", "right_hip_jc_3d"),
         ("spine_base_virtual", "spine_top_virtual"), ("spine_base_virtual", "left_hip_jc_3d"),
         ("spine_base_virtual", "right_hip_jc_3d"), ("spine_top_virtual", "left_shoulder_jc_3d"),
         ("spine_top_virtual", "right_shoulder_jc_3d")]

# Frame markers
frame_indices = np.arange(skeleton_frames.shape[0])
ball_release_frame = df[df['ball_release_frame'] == 0].index.min() if 'ball_release_frame' in df.columns else None
foot_contact_frame = df[df['foot_contact_frame'] == 0].index.min() if 'foot_contact_frame' in df.columns else None

# Velocity chart data prep
athlete_name_chart = df['athlete_name'].iloc[0] if 'athlete_name' in df.columns else "Unknown"
pitcherhand = "LHP" if pitcher_handedness == 'left' else "RHP"
arm_vel = 'left_arm_rotational_velocity_with_respect_to_ground' if pitcherhand == 'LHP' else 'right_arm_rotational_velocity_with_respect_to_ground'
elbow_vel = 'left_elbow_flexion_velocity' if pitcherhand == 'LHP' else 'right_elbow_flexion_velocity'
lead_leg_vel = 'right_knee_extension_velocity' if pitcherhand == 'LHP' else 'left_knee_extension_velocity'
subset_data = df[["handedness", arm_vel, elbow_vel, "time",
                  "trunk_rotational_velocity_with_respect_to_ground",
                  "pelvis_rotational_velocity_with_respect_to_ground",
                  lead_leg_vel, "foot_contact_time", "ball_release_time"]].copy()
subset_data.columns = ["handedness", "arm_velocity", "elbow_velocity", "time",
                       "trunk_velocity", "pelvis_velocity", "lead_leg_extension_velocity",
                       "foot_contact_time", "ball_release_time"]
if pitcherhand == 'LHP':
    subset_data[["pelvis_velocity", "trunk_velocity", "arm_velocity", "lead_leg_extension_velocity"]] *= -1
subset_data["elbow_velocity"] *= -1
foot_contact_time = subset_data.loc[subset_data["foot_contact_time"] == 0.0, "time"].min()
ball_release_time = subset_data.loc[subset_data["ball_release_time"] == 0.0, "time"].min()

# Multi-view rendering
views = {"behind_pitcher": 315, "first_base": 135, "third_base": 225, "home_plate": 45}
for view_name, azim_angle in views.items():
    fig = plt.figure(figsize=(10, 12))
    ax1 = fig.add_subplot(211, projection='3d')
    ax2 = fig.add_subplot(212)
    plt.style.use("dark_background")

    adjusted_azim = 360 - azim_angle if flip_x else azim_angle

    def update(frame):
        ax1.clear(), ax2.clear()
        ax1.set_xlim(x_max, x_min)
        ax1.set_ylim(y_min, y_max)
        ax1.set_zlim(z_min, z_max)
        ax1.view_init(elev=15, azim=adjusted_azim)
        ax1.set_facecolor('black')
        ax1.set_axis_off()

        for j1, j2 in bones:
            if j1 in core_joints and j2 in core_joints:
                a, b = core_joints.index(j1), core_joints.index(j2)
                ax1.plot([skeleton_frames[frame, a, 0], skeleton_frames[frame, b, 0]],
                         [skeleton_frames[frame, a, 1], skeleton_frames[frame, b, 1]],
                         [skeleton_frames[frame, a, 2], skeleton_frames[frame, b, 2]], color='white', linewidth=3)
        for j in range(len(core_joints)):
            x, y, z = skeleton_frames[frame, j]
            v = velocity_normed[frame, j]
            ax1.scatter(x, y, z, color=plt.cm.plasma(v), s=50, edgecolors='black', linewidths=0.5)

        wrist_idx = core_joints.index(thrw_joint)
        for i in range(max(0, frame-10), frame):
            p1, p2 = skeleton_frames[i, wrist_idx], skeleton_frames[i+1, wrist_idx]
            alpha = 0.1 + 0.9 * (i - max(0, frame-10)) / 10
            ax1.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], color='red', linewidth=2, alpha=alpha)

        if ball_release_frame == frame:
            x, y, z = skeleton_frames[frame, wrist_idx]
            ax1.scatter(x, y, z, s=100, c='lime', marker='*')
            ax1.text(x, y, z+0.2, 'Ball Release', color='lime', fontsize=10)

        if foot_contact_frame == frame:
            ankle_joint = "left_ankle_jc_3d" if pitcher_handedness == "right" else "right_ankle_jc_3d"
            if ankle_joint in core_joints:
                ankle_idx = core_joints.index(ankle_joint)
                x, y, z = skeleton_frames[frame, ankle_idx]
                ax1.scatter(x, y, z, s=100, c='orange', marker='^')
                ax1.text(x, y, z+0.2, 'Foot Plant', color='orange', fontsize=10)

        ax1.text2D(0.05, 0.95, f"Frame {frame}", transform=ax1.transAxes, fontsize=12, color='white')

        # Chart rendering
        ax2.set_facecolor("black")
        ax2.plot(subset_data["time"], subset_data["pelvis_velocity"], color="blue", label="Pelvis Velocity")
        ax2.plot(subset_data["time"], subset_data["trunk_velocity"], color="red", label="Trunk Velocity")
        ax2.plot(subset_data["time"], subset_data["arm_velocity"], color="green", label="Arm Velocity")
        ax2.plot(subset_data["time"], subset_data["lead_leg_extension_velocity"], color="white", label="Lead Leg Velocity")
        ax2.plot(subset_data["time"], subset_data["elbow_velocity"], color="gray", label="Elbow Velocity")
        if foot_contact_time is not None:
            ax2.axvline(x=foot_contact_time, color="white", linestyle="dotted", label="Foot Contact")
        if ball_release_time is not None:
            ax2.axvline(x=ball_release_time, color="yellow", linestyle="dotted", label="Ball Release")
        ax2.set_title(f"Rotational Velocities - {pitcherhand} - {athlete_name_chart}", color="white")
        ax2.set_xlabel("Time", color="white")
        ax2.set_ylabel("Angular Velocity", color="white")
        ax2.tick_params(colors="white")
        ax2.legend(facecolor="black", edgecolor="white", labelcolor="white")
        ax2.grid(color="gray", linestyle="--", linewidth=0.5)

    ani = FuncAnimation(fig, update, frames=len(frame_indices), interval=100)
    ani.save(f"pitching_skeleton_with_chart_{view_name}.mp4", writer=FFMpegWriter(fps=10))
    plt.close(fig)

print("✅ Exported all MP4s with rotational velocity chart and 3D skeleton view.")






✅ Exported all MP4s with rotational velocity chart and 3D skeleton view.
